In [1217]:
import pandas as pd
import numpy as np

In [1922]:
#data = pd.read_csv("korean_air_2020_01.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_02.csv")
#data = pd.read_csv("korean_air_2020_03.csv")
#data = pd.read_csv("korean_air_2020_04.csv")
#data = pd.read_csv("korean_air_2020_05.csv")
#data = pd.read_csv("korean_air_2020_6.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_07.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_08.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_09.csv")
#data = pd.read_csv("korean_air_data_korean_air_2020_10.csv")

#data = pd.read_csv("naver_2020_1.csv")
#data = pd.read_csv("naver_2020_2.csv")
#data = pd.read_csv("naver_2020_3.csv")
#data = pd.read_csv("naver_2020_4.csv")

#data = pd.read_csv("naver_2020_5.csv")
#data = pd.read_csv("naver_2020_6.csv")
#data = pd.read_csv("naver_2020_7.csv")
#data = pd.read_csv("naver_2020_8.csv")
data = pd.read_csv("naver_2020_9.csv")

In [1923]:
data = data.dropna()
data.columns = ["a","title","text","repoter","spon", "date"]
date = data.date

In [1924]:
#q불용어 지정
#기자이름
repoter = data["repoter"].unique

In [1925]:
#신문사 이름
spon = data["spon"].unique

In [1926]:
#대표적인 조사
stopwords = ['의','가','이','권','은','들','는','좀','잘','걍','과','도','을','것','를','등','으로','자','에','등','와','한','하다''것','로']

In [1927]:
#불용어 병합
stopwords.extend(data.repoter.values)
stopwords.extend(data.spon.values)

In [1928]:
len(data)

1421

In [1929]:
# 한글 외의 모든 데이터 제거 : 기업명이 영어로 되어있는 경우를 어떻게 처리 해야할까?
data = data.text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer

okt = Okt()
X_train = []
X_train2 =[]

for sen in data :
    temp_X = []
    
    # 토큰화
    temp_X = okt.morphs(sen, stem = True) 
    
    # 불용어 제거
    temp_X = [word for word in temp_X if not word in stopwords] 
    
    #모든 토큰들을 리스트에 기사별로 저장
    X_train.append(temp_X)
    
    #모든 토큰들을  하나의 리스트에 저장
    X_train2.extend(temp_X)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
vocab_size = total_cnt - rare_cnt
print('단어 집합의 크기 :',vocab_size)

단어 집합(vocabulary)의 크기 : 17692
등장 빈도가 2번 이하인 희귀 단어의 수: 7808
단어 집합에서 희귀 단어의 비율: 44.13294144245987
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.7932260156889885
단어 집합의 크기 : 9884


In [1930]:
nouns = []
str = ''

for k in range(0,len(X_train)):
    for i in range(0,len(X_train[k])):
        str = str + X_train[k][i] + ' '
    nouns.append(str)

In [1931]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_df = pd.DataFrame()

tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfidf = tfIdfVectorizer.fit_transform(nouns)

for i in range(0,len(X_train)):
    df = pd.DataFrame(tfidf[i].T.toarray(), index=tfIdfVectorizer.get_feature_names(), columns=["text{}".format(i)])
    tfidf_df = pd.concat([tfidf_df,df],axis = 1)

X_train_tfidf = []
X_train_tfidf2 = []

for i in range(0,len(X_train)):
    idx = tfidf_df['text{}'.format(i)] > 0.01
    result = tfidf_df[idx].index.tolist()
    X_train_tfidf.append(result)
    X_train_tfidf2.extend(result)

In [1932]:
#요일 구하기
import datetime
date2 = []
weekday = []
for x in date:
    x = x.replace("-","/")
    D = x.split("/")
    y = int(D[0])
    y_string = D[0]
    if len(D[1]) < 2 :
        D[1] = "0" + D[1]
        m = int(D[1])
        m_string = D[1]
        
    else : 
        m = int(D[1])
        m_string = D[1]
    
    if len(D[2]) < 2 :
        D[2] = "0" + D[2]
        d = int(D[1])
        d_string = D[2]
        
    else : 
        d = int(D[2])
        d_string = D[2]
        
    date2.append(y_string + "/" + m_string + "/" + d_string)
    weekday.append(['MON','TUE','WED','THU','FRI','SAT','SUN'][datetime.date(y,m,d).weekday()])

In [1933]:
date = date2

In [1934]:
len(date) == len(weekday)

True

In [1935]:
DATE = []
for x in date:
    if x not in DATE:
        DATE.append(x)

In [1936]:
#증감 데이터 불러오기
#increase_r = pd.read_csv("대한항공 대비처리.csv")
increase_r = pd.read_csv("naver 대비처리.csv")

In [1937]:
increase_r 

,Unnamed: 0,년/월/일,종가,대비,거래량(주),거래대금(원),시가,고가,저가,시가총액(백만),상장주식수(주),증감
0,0,2020/12/11,"290,000","4,000","503,588","145,874,384,000","288,000","291,500","287,500","47,636,385","164,263,395",1
1,1,2020/12/10,"286,000","-2,000","959,427","275,249,175,500","288,000","290,000","284,500","46,979,331","164,263,395",0
2,2,2020/12/09,"288,000","3,000","533,304","152,722,372,500","285,500","289,000","285,000","47,307,858","164,263,395",1
3,3,2020/12/08,"285,000","-4,500","566,981","162,586,326,000","291,500","291,500","285,000","46,815,068","164,263,395",-1
4,4,2020/12/07,"289,500","-4,500","821,860","240,096,146,500","300,000","300,500","288,000","47,554,253","164,263,395",-1
...,...,...,...,...,...,...,...,...,...,...,...,...
231,231,2020/01/08,"183,000","-4,000","820,867","149,007,274,500","182,000","183,500","179,000","30,160,851","164,813,395",-1
232,232,2020/01/07,"187,000","6,500","328,624","60,938,161,000","182,500","187,000","181,500","30,820,105","164,813,395",2
233,233,2020/01/06,"180,500","-1,000","287,970","51,850,594,000","180,000","182,000","178,000","29,748,818","164,813,395",0
234,234,2020/01/03,"181,500","-1,000","454,492","82,879,082,000","185,000","186,000","180,500","29,913,631","164,813,395",0


In [1938]:
increase = increase_r[["년/월/일","증감"]]

In [1939]:
weekday_frame = pd.DataFrame( {"date" : date,"weekday" : weekday})

In [1940]:
 for x in range(0,len(weekday_frame)):
    if weekday_frame.weekday[x] == "SAT":
        monday = DATE.index(weekday_frame.date[x])- 2
        weekday_frame.date[x] = DATE[monday]
        weekday_frame.weekday[x] == "MON"
            
    else :
        if weekday_frame.weekday[x] == "SUN":
            monday = DATE.index(weekday_frame.date[x]) - 1
            weekday_frame.date[x] = DATE[monday]
            weekday_frame.weekday[x] == "MON"
                
        else : pass
                

In [1941]:
weekday_frame = weekday_frame.drop_duplicates()

In [1942]:
#weekday_frame.to_csv("weekday_naver.csv")
#weekday_frame.to_csv("weekday_daehan.csv")

In [1943]:
#날자를 중심으로 데이터 병합
increase = increase.rename(columns ={ "년/월/일" : "date"})

In [1944]:
dic_frame = pd.DataFrame({"data" : X_train_tfidf, "date" : date})

In [1945]:
increase2 = pd.merge(weekday_frame, increase , on = "date")

In [1946]:
del increase2["weekday"]

In [1947]:
increase2 = increase2.drop_duplicates()

In [1948]:
dic_frame2 = pd.merge( dic_frame, increase2, on = "date", how = "left")

In [1949]:
dic = dic_frame2.fillna(0)

In [1950]:
#딕셔너리 key 생성
key_list = []
for v in X_train_tfidf2:
    if v not in key_list:
        key_list.append(v)

In [1951]:
#예상 소요 시간 300분
#사전 만들기
dictionary = {"<UNK>" : { "횟수" : 0, "점수" : 0, "극성정도" : 0}}

for k in key_list:  #키 리스트안에 있는 단어가 언급이 되는 경우
    count = 0
    score = 0
    for x in range(0,len(dic)):  #한 기사씩 검사한다.
        d = dic.loc[x,:].data
        
        
        #count 는 전체 언급된 횟수, score는 증감의 총합, pority는 총합의 평균
        if k in d:
            count = count + d.count(k)
            score = score + (dic.loc[x,:]["증감"] * d.count(k))
        
       
        #elif k not in word_list:
        else:
            pass
    try:
        polarity  = score/count
        
    except ZeroDivisionError as e:
        print(x,k)
    dictionary[k] = { "횟수" : count, "점수" : score, "극성정도" : polarity}  

In [1952]:
#기사별 점수 계산

from collections import Counter
scores = []
miss = []
for x in range (0, len(dic)):
    key = set(dic.data[x])
    score = 0
    count = 0
    for i in key:
        score += (dictionary[i]["극성정도"] * Counter(dic.data[x])[i])
        count += Counter(dic.data[x])[i]
    try :
        score = score/count
        scores.append(score)
        
    except ZeroDivisionError as e:
        print(x,k)
        miss.append(x)

In [1953]:
#이상치 제거
dic.drop(miss, inplace=True)
dic["score"] = scores

In [1954]:
#날자별 점수 평균계산
score_data = dic.groupby(['date'])["score"].mean()

In [1955]:
#날자별 평균 데이터 저장
score_data.to_csv("naver_09_score_data.csv")

In [1956]:
dictionary

{'<UNK>': {'횟수': 0, '점수': 0, '극성정도': 0},
 '가장': {'횟수': 1421, '점수': -347.0, '극성정도': -0.2441942294159043},
 '가지다': {'횟수': 7, '점수': 0.0, '극성정도': 0.0},
 '각종': {'횟수': 8, '점수': 0.0, '극성정도': 0.0},
 '격려': {'횟수': 5, '점수': 0.0, '극성정도': 0.0},
 '겪다': {'횟수': 8, '점수': 0.0, '극성정도': 0.0},
 '경기도': {'횟수': 3, '점수': 0.0, '극성정도': 0.0},
 '경제': {'횟수': 700, '점수': -381.0, '극성정도': -0.5442857142857143},
 '공공': {'횟수': 3, '점수': 0.0, '극성정도': 0.0},
 '곽상욱': {'횟수': 5, '점수': 0.0, '극성정도': 0.0},
 '관계': {'횟수': 3, '점수': 0.0, '극성정도': 0.0},
 '구입': {'횟수': 5, '점수': 0.0, '극성정도': 0.0},
 '기관': {'횟수': 1347, '점수': -347.0, '극성정도': -0.25760950259836674},
 '기자': {'횟수': 1421, '점수': -347.0, '극성정도': -0.2441942294159043},
 '네이버': {'횟수': 1421, '점수': -347.0, '극성정도': -0.2441942294159043},
 '다방면': {'횟수': 3, '점수': 0.0, '극성정도': 0.0},
 '대상': {'횟수': 1421, '점수': -347.0, '극성정도': -0.2441942294159043},
 '대웅': {'횟수': 3, '점수': 0.0, '극성정도': 0.0},
 '대응': {'횟수': 9, '점수': 0.0, '극성정도': 0.0},
 '도입': {'횟수': 784, '점수': -52.0, '극성정도': -0.0663265306122449},
 '돼다

In [1957]:
import pickle

In [1958]:
#딕셔너리 저장
# save data
with open('naver_09','wb') as fw:
    pickle.dump(dictionary, fw)

In [1959]:
import winsound
winsound.PlaySound("SystemExit", winsound.SND_ALIAS)